# Data Exploration
## Demo: Titanic Data 

We will implement the following procedure:
1. Prepare the environment by importing the major libraries, which contain modules and functions we will need.
2. Get the available data
3. Explore the data to get an impression of what does it contain.
4. Clean the data, so it can be further analysed

# Environment

In [5]:
# import pandas for structuring the data
import pandas as pd

# import numpy for numerical analysis
import numpy as np

# import libs for diagrams inline with the text
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# other utilities
from sklearn import datasets, preprocessing, metrics

# Data Input
In this demo we will use an excel file with the original data. In many other cases we can only find plain text or csv files.

In [18]:
!pip install xlrd

In [34]:
# read the Excel file from your data folder into a data frame
df = pd.read_csv('TitanicData.csv', index_col=None, na_values=['NA'])

In [36]:
# see the size
df.shape

(1310, 14)

In [38]:
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [40]:
# see which are the attribute labels
list(df)

['pclass',
 'survived',
 'name',
 'sex',
 'age',
 'sibsp',
 'parch',
 'ticket',
 'fare',
 'cabin',
 'embarked',
 'boat',
 'body',
 'home.dest']

In [42]:
# see the first five records
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# Data Cleaning and Preparation

In [184]:
# count the missing values
df.isnull().sum()

pclass       1
survived     1
name         1
sex          1
age          0
sibsp        1
parch        1
ticket       1
fare         0
embarked     0
home.dest    0
dtype: int64

### Delete Rows and Columns

In [48]:
# remove most empty columns, which are not so informative
df = df.drop(['body', 'cabin', 'boat'], axis=1)

In [50]:
df.shape

(1310, 11)

### Replace with Average

In [53]:
# replace the missing age with the average age
mean_age=df.age.mean()
df['age']= df['age'].fillna(mean_age)

In [55]:
# see the current number of data
df.count()


pclass       1309
survived     1309
name         1309
sex          1309
age          1310
sibsp        1309
parch        1309
ticket       1309
fare         1308
embarked     1307
home.dest     745
dtype: int64

In [57]:
# fill the missing home destination with 'NA'
df["home.dest"] = df["home.dest"].fillna("NA")
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,S,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"


In [59]:
# see the current state of null values
df.isnull().sum()

pclass       1
survived     1
name         1
sex          1
age          0
sibsp        1
parch        1
ticket       1
fare         2
embarked     3
home.dest    0
dtype: int64

In [61]:
# replace the  missing fare values with the average
mean_fare = df.fare.mean()
mean_fare

33.29547928134557

In [63]:
df['fare'] = df['fare'].fillna(mean_fare)

### Replace with Mode

In [66]:
# find the most used 'embarked' value
mode_emb = df.embarked.mode()
mode_emb

0    S
Name: embarked, dtype: object

In [68]:
# replace the missing embarked values with the mode
df['embarked']=df['embarked'].fillna('S')

### Transform Categorical Data into Numeric

As a preprocessing, we will convert the strings into integer keys, making it easier for the  algorithms to find patterns. 
- “Female” and “Male” are categorical values and will be converted to 0 and 1 respectively
- The “name”, “ticket”, and “home.dest” columns consist of non-categorical string values, which are difficult to use in our algorithm, so we will drop them from the data set

In [72]:
# define a function for transformation
def preprocessor(df):
    processed_df = df.copy()
    le = preprocessing.LabelEncoder()
    processed_df['sex'] = le.fit_transform(df['sex'])
    processed_df['embarked'] = le.fit_transform(df['embarked'])
    processed_df = processed_df.drop(['name','ticket','home.dest'], axis=1)
    return processed_df

In [74]:
# call the transformation function
dfp = preprocessor(df)

In [76]:
dfps = pd.get_dummies(dfp, columns = ['sex'])
dfps

,pclass,survived,age,sibsp,parch,fare,embarked,sex_0,sex_1,sex_2
0,1.0,1.0,29.000000,0.0,0.0,211.337500,2,True,False,False
1,1.0,1.0,0.916700,1.0,2.0,151.550000,2,False,True,False
2,1.0,0.0,2.000000,1.0,2.0,151.550000,2,True,False,False
3,1.0,0.0,30.000000,1.0,2.0,151.550000,2,False,True,False
4,1.0,0.0,25.000000,1.0,2.0,151.550000,2,True,False,False
...,...,...,...,...,...,...,...,...,...,...
1305,3.0,0.0,29.881135,1.0,0.0,14.454200,0,True,False,False
1306,3.0,0.0,26.500000,0.0,0.0,7.225000,0,False,True,False
1307,3.0,0.0,27.000000,0.0,0.0,7.225000,0,False,True,False
1308,3.0,0.0,29.000000,0.0,0.0,7.875000,2,False,True,False


In [78]:
dfps.head()

,pclass,survived,age,sibsp,parch,fare,embarked,sex_0,sex_1,sex_2
0,1.0,1.0,29.0000,0.0,0.0,211.3375,2,True,False,False
1,1.0,1.0,0.9167,1.0,2.0,151.5500,2,False,True,False
2,1.0,0.0,2.0000,1.0,2.0,151.5500,2,True,False,False
3,1.0,0.0,30.0000,1.0,2.0,151.5500,2,False,True,False
4,1.0,0.0,25.0000,1.0,2.0,151.5500,2,True,False,False


In [80]:
dfp.shape

(1310, 8)

In [82]:
# dfp = np.nan_to_num(dfp)

In [84]:
np.all(np.isfinite(dfp))

False

In [86]:
np.any(np.isnan(dfp))

True

# Data Exploration

In [89]:
# see the types of the attributes
df.dtypes

pclass       float64
survived     float64
name          object
sex           object
age          float64
sibsp        float64
parch        float64
ticket        object
fare         float64
embarked      object
home.dest     object
dtype: object

In [91]:
# get some insights of the value scope
df.describe()

,pclass,survived,age,sibsp,parch,fare
count,1309.000000,1309.000000,1310.000000,1309.000000,1309.000000,1310.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479
std,0.837836,0.486055,12.878277,1.041658,0.865560,51.719113
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,29.881135,0.000000,0.000000,14.454200
75%,3.000000,1.000000,35.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


### Measures of Central Tendency

In [94]:
# mean
# np.mean(df[['age']])
np.mean(df.age)

29.881134512428297

In [96]:
# the average in groups
df.groupby("pclass")["age"].mean()

pclass
1.0    38.039569
2.0    29.528333
3.0    26.302223
Name: age, dtype: float64

In [98]:
# weighted mean
x=np.average(df.age, weights=df.pclass)

In [100]:
# trimmed mean - ignores the 10% extream values from both ends (deciles)
from scipy import stats as st
st.trim_mean(df.age, 0.1)

29.37415983900865

### Measures of Variability

In [103]:
# standard deviation
df.age.std()

12.878277095207078

In [105]:
# quantiles
df.age.quantile(0.75)

35.0

In [107]:
# quantiles
df.age.quantile([0.05, 0.25, 0.50, 0.75, 0.95])

0.05     7.450000
0.25    22.000000
0.50    29.881135
0.75    35.000000
0.95    55.000000
Name: age, dtype: float64

### Split in Bins

In [110]:
# Did the age matter?
bins = [0,10,20,30,40,50,60,70,80] 

In [112]:
age=df.groupby([(pd.cut(df.age, bins))]).count()
age

C:\Users\baban\AppData\Local\Temp\ipykernel_14864\666366791.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age=df.groupby([(pd.cut(df.age, bins))]).count()


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,home.dest
age,,,,,,,,,,,
"(0, 10]",86,86,86,86,86,86,86,86,86,86,86
"(10, 20]",162,162,162,162,162,162,162,162,162,162,162
"(20, 30]",624,624,624,624,625,624,624,624,625,625,625
"(30, 40]",210,210,210,210,210,210,210,210,210,210,210
"(40, 50]",132,132,132,132,132,132,132,132,132,132,132
"(50, 60]",62,62,62,62,62,62,62,62,62,62,62
"(60, 70]",27,27,27,27,27,27,27,27,27,27,27
"(70, 80]",6,6,6,6,6,6,6,6,6,6,6


In [114]:
plt.hist(df['age'], bins=20)

(array([ 51.,  21.,  22.,  40., 114., 160., 128., 386.,  97.,  64.,  51.,
         57.,  38.,  27.,  21.,  20.,   5.,   5.,   2.,   1.]),
 array([ 0.1667  ,  4.158365,  8.15003 , 12.141695, 16.13336 , 20.125025,
        24.11669 , 28.108355, 32.10002 , 36.091685, 40.08335 , 44.075015,
        48.06668 , 52.058345, 56.05001 , 60.041675, 64.03334 , 68.025005,
        72.01667 , 76.008335, 80.      ]),
 <BarContainer object of 20 artists>)

In [116]:
labels = ['young', 'middle aged', 'old']

In [118]:
age=df.groupby([(pd.cut(df.age, 3, labels))]).count()

C:\Users\baban\AppData\Local\Temp\ipykernel_14864\1438278010.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age=df.groupby([(pd.cut(df.age, 3, labels))]).count()


In [120]:
age

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,home.dest
age,,,,,,,,,,,
"(0.0869, 26.778]",474,474,474,474,474,474,474,474,474,474,474
"(26.778, 53.389]",758,758,758,758,759,758,758,758,759,759,759
"(53.389, 80.0]",77,77,77,77,77,77,77,77,77,77,77


## Plotting

### Box Plot

In [124]:
# box plot
# here we see the median25th and 75th percentiles, the range, and the outliers
df.age.plot.box()

<Axes: >

In [126]:
# categorical data vs numeric data
df.boxplot(by='embarked', column='age')

<Axes: title={'center': 'age'}, xlabel='embarked'>

### Histogram

In [129]:
df.age.plot.hist()

<Axes: title={'center': 'age'}, ylabel='Frequency'>

### Density Plot

In [132]:
# parameters can control the smoothness
df.age.plot.hist(density=True)
df.age.plot.density()

<Axes: title={'center': 'age'}, ylabel='Density'>

### Bar Charts

In [135]:
# Non-numeric data is not included in the statistic above, but can be plotted
df['embarked'].value_counts().plot(kind='bar')

<Axes: title={'center': 'age'}, xlabel='embarked', ylabel='Density'>

In [137]:
# Numeric data can also be plotted 
df['survived'].value_counts().plot(kind='bar')

<Axes: title={'center': 'age'}, xlabel='survived', ylabel='Density'>

### Scatterplot

In [140]:
df.plot.scatter(x='fare', y='age', figsize=(6, 6), marker = '$\u25EF$')

<Axes: xlabel='fare', ylabel='age'>

In [141]:
sns.kdeplot(df.sibsp)

<Axes: xlabel='fare', ylabel='age'>

In [143]:
# a topographical map, presents the density of points of two variables
sns.kdeplot(df.age)

<Axes: xlabel='fare', ylabel='age'>

### Violin Plot

In [147]:
# enhancement to the boxplot
# shos the dencity estimates
sns.violinplot(df.embarked, inner="quartile", color="white")

<Axes: xlabel='fare', ylabel='age'>

In [149]:
sns.violinplot(df.age, inner="quartile", color="white")

<Axes: xlabel='fare', ylabel='age'>

In [151]:
#create histogram to visualize values in dataset
df.hist()

array([[<Axes: title={'center': 'pclass'}>,
        <Axes: title={'center': 'survived'}>],
       [<Axes: title={'center': 'age'}>,
        <Axes: title={'center': 'sibsp'}>],
       [<Axes: title={'center': 'parch'}>,
        <Axes: title={'center': 'fare'}>]], dtype=object)

In [152]:
# plot all observations to discover outliers
df.plot.scatter(x='age', y='fare', c='pclass', colormap='viridis')

<Axes: xlabel='age', ylabel='fare'>

In [154]:
# z-score filter of outliers: (z-score < 3*std) in a column
from scipy import stats
import numpy

Z-score is a statistical measurement that describes a value's relationship to the mean of a group of values. To spot outliers with the z-score, set a threshold—commonly 3. This means any z-score beyond 3 or below -3 is an outlier. It captures about 0.3% of data in a normal distribution.

In [158]:
z_age = stats.zscore(df['age'])
z_age

0      -6.844635e-02
1      -2.249951e+00
2      -2.165801e+00
3       9.233446e-03
4      -3.791655e-01
            ...     
1305    2.759741e-16
1306   -2.626458e-01
1307   -2.238059e-01
1308   -6.844635e-02
1309    2.759741e-16
Name: age, Length: 1310, dtype: float64

In [160]:
z_fare = stats.zscore(df['fare'])
z_fare

0       3.443795e+00
1       2.287349e+00
2       2.287349e+00
3       2.287349e+00
4       2.287349e+00
            ...     
1305   -3.644393e-01
1306   -5.042707e-01
1307   -5.042707e-01
1308   -4.916981e-01
1309    1.374374e-16
Name: fare, Length: 1310, dtype: float64

In [162]:
dfa = df[np.abs(z_age) < 3]
dfa

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.000000,0.0,0.0,24160,211.337500,S,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.916700,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.000000,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.000000,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.000000,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...
1305,3.0,0.0,"Zabour, Miss. Thamine",female,29.881135,1.0,0.0,2665,14.454200,C,NA
1306,3.0,0.0,"Zakarian, Mr. Mapriededer",male,26.500000,0.0,0.0,2656,7.225000,C,NA
1307,3.0,0.0,"Zakarian, Mr. Ortin",male,27.000000,0.0,0.0,2670,7.225000,C,NA
1308,3.0,0.0,"Zimmerman, Mr. Leo",male,29.000000,0.0,0.0,315082,7.875000,S,NA


In [164]:
dff = df[np.abs(z_fare) < 3]
dff

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,home.dest
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.916700,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.000000,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.000000,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.000000,1.0,2.0,113781,151.550000,S,"Montreal, PQ / Chesterville, ON"
5,1.0,1.0,"Anderson, Mr. Harry",male,48.000000,0.0,0.0,19952,26.550000,S,"New York, NY"
...,...,...,...,...,...,...,...,...,...,...,...
1305,3.0,0.0,"Zabour, Miss. Thamine",female,29.881135,1.0,0.0,2665,14.454200,C,NA
1306,3.0,0.0,"Zakarian, Mr. Mapriededer",male,26.500000,0.0,0.0,2656,7.225000,C,NA
1307,3.0,0.0,"Zakarian, Mr. Ortin",male,27.000000,0.0,0.0,2670,7.225000,C,NA
1308,3.0,0.0,"Zimmerman, Mr. Leo",male,29.000000,0.0,0.0,315082,7.875000,S,NA


In [166]:
dff.plot.scatter(x='age', y='fare', c='pclass', colormap='viridis')

<Axes: xlabel='age', ylabel='fare'>

In [167]:
# visualise the features and the response using scatterplots
sns.pairplot(dff, x_vars=['age'], y_vars='fare', height=5, aspect=0.8)

In [170]:
x = numpy.quantile(df['age'], [0,0.25,0.5,0.75,1])
print(x)

[ 0.1667     22.         29.88113451 35.         80.        ]


### 3D Skatter Plot

In [173]:
# for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio

In [174]:
# 3D scatter plot
fig = px.scatter_3d(df, x="age", y="sex", z='fare', color="pclass", size='pclass', size_max=20, opacity=0.8)
fig.show()

ValueError: 
    Invalid element(s) received for the 'size' property of scatter3d.marker
        Invalid elements include: [nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

## Correlation

In [ ]:
# get the numeric data
df.select_dtypes(include=[np.number]).columns

In [ ]:
ndf = df.select_dtypes(include=[np.number])

In [ ]:
# Correlation matrix
corrmat = ndf.corr()
corrmat

In [ ]:
sns.heatmap(corrmat, annot=True)
plt.show()

## Reference
https://www.kaggle.com/c/titanic/data <br>
https://blog.socialcops.com/technology/data-science/machine-learning-python/<br>
https://www.youtube.com/watch?v=siEPqQsPLKA